In this notebook, we will explore various data cleaning techniques and apply them to a dataset.

We'll cover handling missing data, duplicates, formatting issues, outliers, data validation, and more.


In [6]:
## Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
## Loading the Dataset
# For now, we'll create a sample dataset with placeholder values.
data = pd.DataFrame({
    'Name': ['John', 'Alice', 'Bob', 'Cathy', 'David', 'David', 'Elon'],
    'Age': [25, np.nan, 30, 35, 40, 40, 50],
    'Salary': [50000, 60000, 55000, 70000, 80000, 80000, 300000000],
    'City': ['New York', 'London', 'Paris', 'New York', 'London', 'London', 'Los Angeles']
})

In [8]:
data.head(10)

,Name,Age,Salary,City
0,John,25.0,50000,New York
1,Alice,NaN,60000,London
2,Bob,30.0,55000,Paris
3,Cathy,35.0,70000,New York
4,David,40.0,80000,London
5,David,40.0,80000,London
6,Elon,50.0,300000000,Los Angeles


## Handling Missing Data

In [9]:
data.isnull().sum()

Name      0
Age       1
Salary    0
City      0
dtype: int64

### 1. Deletion

In [10]:
data_deleted = data.dropna()
data_deleted.head()

,Name,Age,Salary,City
0,John,25.0,50000,New York
2,Bob,30.0,55000,Paris
3,Cathy,35.0,70000,New York
4,David,40.0,80000,London
5,David,40.0,80000,London


### 2. Imputation (Simple)

In [11]:
data_imputed_mean = data.fillna(data["Age"].mean())
data_imputed_mean.head()

,Name,Age,Salary,City
0,John,25.000000,50000,New York
1,Alice,36.666667,60000,London
2,Bob,30.000000,55000,Paris
3,Cathy,35.000000,70000,New York
4,David,40.000000,80000,London


In [12]:
data_imputed_median = data.fillna(data["Age"].median())
data_imputed_median.head()

,Name,Age,Salary,City
0,John,25.0,50000,New York
1,Alice,37.5,60000,London
2,Bob,30.0,55000,Paris
3,Cathy,35.0,70000,New York
4,David,40.0,80000,London


In [13]:
data_imputed_mode = data.fillna(data["Age"].mode()[0])
data_imputed_mode.head()

,Name,Age,Salary,City
0,John,25.0,50000,New York
1,Alice,40.0,60000,London
2,Bob,30.0,55000,Paris
3,Cathy,35.0,70000,New York
4,David,40.0,80000,London


### 3. Imputation (Advanced)

In [14]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)

In [15]:
# Impute the age column
data_imputed_knn = imputer.fit_transform(data[['Age']])

# Add the imputed values to the original dataframe
data['Age'] = data_imputed_knn
data.head()

,Name,Age,Salary,City
0,John,25.000000,50000,New York
1,Alice,36.666667,60000,London
2,Bob,30.000000,55000,Paris
3,Cathy,35.000000,70000,New York
4,David,40.000000,80000,London


In [16]:
data.isnull().sum()

Name      0
Age       0
Salary    0
City      0
dtype: int64

## Dealing with Duplicates

### Identifying duplicates

In [17]:
data.head()

,Name,Age,Salary,City
0,John,25.000000,50000,New York
1,Alice,36.666667,60000,London
2,Bob,30.000000,55000,Paris
3,Cathy,35.000000,70000,New York
4,David,40.000000,80000,London


In [18]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5     True
6    False
dtype: bool

### Removing duplicates

In [19]:
data_deduplicated = data.drop_duplicates()
data_deduplicated.head()

,Name,Age,Salary,City
0,John,25.000000,50000,New York
1,Alice,36.666667,60000,London
2,Bob,30.000000,55000,Paris
3,Cathy,35.000000,70000,New York
4,David,40.000000,80000,London


In [20]:
data = data_deduplicated

## Data Formatting and Standardization

### Text case inconsistencies

In [21]:
data['City'] = data['City'].str.lower()

In [22]:
data.head()

,Name,Age,Salary,City
0,John,25.000000,50000,new york
1,Alice,36.666667,60000,london
2,Bob,30.000000,55000,paris
3,Cathy,35.000000,70000,new york
4,David,40.000000,80000,london


### Inconsistent value representations

In [23]:
data['City'] = data['City'].replace({'new york': 'NY', 'london': 'LDN', 'paris': 'PAR'})

In [24]:
data.head()

,Name,Age,Salary,City
0,John,25.000000,50000,NY
1,Alice,36.666667,60000,LDN
2,Bob,30.000000,55000,PAR
3,Cathy,35.000000,70000,NY
4,David,40.000000,80000,LDN


## Handling Outliers

### Identifying outliers using z-scores

In [25]:
from scipy import stats

In [26]:
z_scores = np.abs(stats.zscore(data['Salary']))
threshold = 2
outliers = np.where(z_scores > threshold)
data.iloc[outliers]

,Name,Age,Salary,City
6,Elon,50.0,300000000,los angeles


### Handling outliers using winsorization

In [27]:
data_winsorized = data.copy()
data_winsorized['Salary'] = stats.mstats.winsorize(data_winsorized['Salary'], limits=0.2)

In [28]:
data_winsorized.head(6)

,Name,Age,Salary,City
0,John,25.000000,55000,NY
1,Alice,36.666667,60000,LDN
2,Bob,30.000000,55000,PAR
3,Cathy,35.000000,70000,NY
4,David,40.000000,80000,LDN
6,Elon,50.000000,80000,los angeles


In [29]:
# Remove the outliers from the salary column
data = data[(z_scores < threshold)]

In [30]:
data.head(10)

,Name,Age,Salary,City
0,John,25.000000,50000,NY
1,Alice,36.666667,60000,LDN
2,Bob,30.000000,55000,PAR
3,Cathy,35.000000,70000,NY
4,David,40.000000,80000,LDN


## Data Transformation and Feature Engineering

In [31]:
# Scaling and normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['Salary_Normalized'] = scaler.fit_transform(data[['Salary']])
data.head()

/var/folders/3q/r2zc74911hn2f967w656zszm0000gn/T/ipykernel_75933/572325590.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Salary_Normalized'] = scaler.fit_transform(data[['Salary']])


,Name,Age,Salary,City,Salary_Normalized
0,John,25.000000,50000,NY,0.000000
1,Alice,36.666667,60000,LDN,0.333333
2,Bob,30.000000,55000,PAR,0.166667
3,Cathy,35.000000,70000,NY,0.666667
4,David,40.000000,80000,LDN,1.000000


In [32]:
# Standardize the age column
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data['Age_Standardized'] = scaler.fit_transform(data[['Age']])
data.head()

/var/folders/3q/r2zc74911hn2f967w656zszm0000gn/T/ipykernel_75933/2725307977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Age_Standardized'] = scaler.fit_transform(data[['Age']])


,Name,Age,Salary,City,Salary_Normalized,Age_Standardized
0,John,25.000000,50000,NY,0.000000,-1.581139
1,Alice,36.666667,60000,LDN,0.333333,0.632456
2,Bob,30.000000,55000,PAR,0.166667,-0.632456
3,Cathy,35.000000,70000,NY,0.666667,0.316228
4,David,40.000000,80000,LDN,1.000000,1.264911


### Encoding categorical variables

In [33]:
# One-hot encoding for the city column
data = pd.get_dummies(data, columns=['City'])
data.head()

,Name,Age,Salary,Salary_Normalized,Age_Standardized,City_LDN,City_NY,City_PAR
0,John,25.000000,50000,0.000000,-1.581139,False,True,False
1,Alice,36.666667,60000,0.333333,0.632456,True,False,False
2,Bob,30.000000,55000,0.166667,-0.632456,False,False,True
3,Cathy,35.000000,70000,0.666667,0.316228,False,True,False
4,David,40.000000,80000,1.000000,1.264911,True,False,False


Our dataset is now clean and ready for analysis. Some decisions could have been made differently, and the cleaning process is not always straightforward. However, the techniques we've covered are a good starting point for most datasets.

Keep in mind that the cleaning process affects the results of any analysis or machine learning model. Therefore, it's important to understand the data and the context in which it was collected to make informed decisions about how to handle missing data, duplicates, outliers, and other issues.